In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ElhubCassandraIntegration") \
    .master("local[*]") \
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.5.1") \
    .config("spark.cassandra.connection.host", "127.0.0.1") \
    .config("spark.cassandra.connection.port", "9042") \
    .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions") \
    .config("spark.sql.catalog.mycatalog", "com.datastax.spark.connector.datasource.CassandraCatalog") \
    .config("spark.cassandra.connection.keepAliveMS", "60000") \
    .config("spark.cassandra.output.consistency.level", "ONE") \
    .getOrCreate()

print("✅ SparkSession started with Cassandra integration")

25/10/09 14:03:56 WARN Utils: Your hostname, Fabians-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.20.6.230 instead (on interface en0)
25/10/09 14:03:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/fabianheflo/.ivy2/cache
The jars for the packages stored in: /Users/fabianheflo/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5fb45ff1-2455-4f1f-90d7-0d1b51bb17b5;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.5.1 in central


:: loading settings :: url = jar:file:/Users/fabianheflo/UNI_courses/IND320/IND320/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.5.1 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.11.0 in central
	found org.apache.cassandra#java-driver-core-shaded;4.18.1 in central
	found com.datastax.oss#native-protocol;1.5.1 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found org.apache.cassandra#java-driver-mapper-runtime;4.18.1 in central
	found org.apache.cassandra#java-driver-query-builder;4.18.1 in central
	found org.apache.commons#commons-lang3;3.10 in central
	found com.thoughtworks.paranamer#paranamer;2.8 in central
	found org.scala-lang#scala-reflect;2.12.19 in central
:: resolution report :: resolve 250ms :: artifacts dl 

✅ SparkSession started with Cassandra integration


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 51717)
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.12/3.12.9/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/homebrew/Cellar/python@3.12/3.12.9/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
  File "/opt/homebrew/Cellar/python@3.12/3.12.9/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socketserver.py", line 362, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/homebrew/Cellar/python@3.12/3.12.9/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socketserver.py", line 766, in __init__
    self.handle()
  File "/Users/fabianheflo/UNI_courses/IND320/IND320/.venv/lib/py

In [2]:
from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

session.execute("""
CREATE KEYSPACE IF NOT EXISTS elhub_data
WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '1'}
""")

session.execute("""
CREATE TABLE IF NOT EXISTS elhub_data.production_raw (
    meteringgridarea TEXT,
    time_start TIMESTAMP,
    productiontype TEXT,
    businesstype TEXT,
    quantity DOUBLE,
    resolution TEXT,
    resourceid TEXT,
    time_end TIMESTAMP,
    PRIMARY KEY (meteringgridarea, time_start, productiontype)
)
""")

print("✅ Keyspace og tabell opprettet")

✅ Keyspace og tabell opprettet


### Testskriv én rad

In [ ]:
from datetime import datetime

data = [("NO1", datetime.utcnow(), "Hydro", "A04", 123.45, "PT1H", "RES001", datetime.utcnow())]
columns = ["meteringgridarea", "time_start", "productiontype", "businesstype", 
           "quantity", "resolution", "resourceid", "time_end"]

df = spark.createDataFrame(data, columns)

df.write.format("org.apache.spark.sql.cassandra") \
    .mode("append") \
    .options(table="production_raw", keyspace="elhub_data") \
    .save()

print("✅ Data skrevet til Cassandra!")

/var/folders/db/2wck1jtn25v9rddfx0kw90qr0000gn/T/ipykernel_13351/523128196.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  data = [("NO1", datetime.utcnow(), "Hydro", "A04", 123.45, "PT1H", "RES001", datetime.utcnow())]


✅ Data skrevet til Cassandra!


### Test lesing tilbake fra Cassandra

In [4]:
df_read = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="production_raw", keyspace="elhub_data") \
    .load()

df_read.show()

+----------------+--------------------+--------------+------------+--------+----------+----------+--------------------+
|meteringgridarea|          time_start|productiontype|businesstype|quantity|resolution|resourceid|            time_end|
+----------------+--------------------+--------------+------------+--------+----------+----------+--------------------+
|             NO1| 2024-10-08 02:00:00|         Solar|         A04|  123.45|      PT1H|    RES001| 2024-10-08 03:00:00|
|             NO1|2025-10-09 11:58:...|         Hydro|         A04|  123.45|      PT1H|    RES001|2025-10-09 11:58:...|
|             NO1|2025-10-09 12:04:...|         Hydro|         A04|  123.45|      PT1H|    RES001|2025-10-09 12:04:...|
+----------------+--------------------+--------------+------------+--------+----------+----------+--------------------+



25/10/09 14:38:35 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 902266 ms exceeds timeout 120000 ms
25/10/09 14:38:35 WARN SparkContext: Killing executors is not supported by current scheduler.
25/10/09 14:38:41 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

# OLD

In [1]:
from cassandra.cluster import Cluster

cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

In [2]:
session.execute("""
CREATE KEYSPACE IF NOT EXISTS elhub_data
WITH REPLICATION = {
  'class': 'SimpleStrategy',
  'replication_factor': 1
};
""")

print("✅ Keyspace 'elhub_data' opprettet!")

✅ Keyspace 'elhub_data' opprettet!


In [3]:
session.execute("USE elhub_data;")

session.execute("""
CREATE TABLE IF NOT EXISTS production_raw (
    meteringgridarea text,
    productiontype text,
    resolution text,
    time_start timestamp,
    time_end timestamp,
    quantity double,
    businesstype text,
    resourceid text,
    PRIMARY KEY ((meteringgridarea), time_start, productiontype)
);
""")

print("✅ Tabell 'production_raw' opprettet!")


✅ Tabell 'production_raw' opprettet!


In [4]:
from datetime import datetime

session.execute("""
INSERT INTO production_raw (meteringgridarea, productiontype, resolution, time_start, time_end, quantity, businesstype, resourceid)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
""", (
    "NO1", "Solar", "PT1H",
    datetime(2024, 10, 8, 0, 0, 0),
    datetime(2024, 10, 8, 1, 0, 0),
    123.45, "A04", "RES001"
))

rows = session.execute("SELECT * FROM production_raw;")
for row in rows:
    print(row)


Row(meteringgridarea='NO1', time_start=datetime.datetime(2024, 10, 8, 0, 0), productiontype='Solar', businesstype='A04', quantity=123.45, resolution='PT1H', resourceid='RES001', time_end=datetime.datetime(2024, 10, 8, 1, 0))


In [ ]:
import os
os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages com.datastax.spark:spark-cassandra-connector_2.12:3.4.1 pyspark-shell"
)


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ElhubCassandraTest") \
    .master("local[*]") \
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.4.1") \
    .config("spark.cassandra.connection.host", "127.0.0.1") \
    .config("spark.cassandra.connection.port", "9042") \
    .config("spark.cassandra.output.consistency.level", "ONE") \
    .config("spark.cassandra.connection.keep_alive_ms", "60000") \
    .getOrCreate()

print("✅ SparkSession restarted and connected")

# Eksempeldata
data = [
    ("NO2", "Wind", "PT1H", "2024-10-08T02:00:00Z", "2024-10-08T03:00:00Z", 200.5, "A04", "RES002"),
]

columns = [
    "meteringgridarea", "productiontype", "resolution", "time_start",
    "time_end", "quantity", "businesstype", "resourceid"
]

df = spark.createDataFrame(data, columns)

# Skriv til Cassandra
df.write \
    .format("org.apache.spark.sql.cassandra") \
    .mode("append") \
    .options(table="production_raw", keyspace="elhub_data") \
    .save()

print("✅ Skrevet til Cassandra!")

# Les tilbake
df_read = spark.read \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="production_raw", keyspace="elhub_data") \
    .load()

df_read.show()


In [5]:
from pyspark.sql import SparkSession

# ⚙️ Bygg SparkSession
spark = SparkSession.builder \
    .appName("ElhubCassandraIntegration") \
    .master("local[*]") \
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.5.1") \
    .config("spark.cassandra.connection.host", "127.0.0.1") \
    .config("spark.cassandra.connection.port", "9042") \
    .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions") \
    .config("spark.sql.catalog.mycatalog", "com.datastax.spark.connector.datasource.CassandraCatalog") \
    .config("spark.cassandra.output.consistency.level", "ONE") \
    .config("spark.cassandra.connection.keepAliveMS", "60000") \
    .getOrCreate()

print("✅ SparkSession started with Cassandra integration")

# 🔍 Testforbindelse — les fra systemtabellen i Cassandra
try:
    # Les en eksisterende tabell i ditt eget keyspace
    ping_df = spark.read \
        .format("org.apache.spark.sql.cassandra") \
        .options(table="production_raw", keyspace="elhub_data") \
        .load()

    ping_df.show(5)

except Exception as e:
    print("❌ Feil ved lesing fra Cassandra:")
    print(e)

25/10/09 13:59:59 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/10/09 13:59:59 WARN DeprecatedConfigParameter: spark.cassandra.connection.keep_alive_ms is deprecated (DSE 6.0.0) and has been automatically replaced with parameter spark.cassandra.connection.keepAliveMS. 
25/10/09 13:59:59 WARN DeprecatedConfigParameter: spark.cassandra.connection.keep_alive_ms is deprecated (DSE 6.0.0) and has been automatically replaced with parameter spark.cassandra.connection.keepAliveMS. 
25/10/09 13:59:59 WARN DeprecatedConfigParameter: spark.cassandra.connection.keep_alive_ms is deprecated (DSE 6.0.0) and has been automatically replaced with parameter spark.cassandra.connection.keepAliveMS. 


✅ SparkSession started with Cassandra integration
+----------------+--------------------+--------------+------------+--------+----------+----------+--------------------+
|meteringgridarea|          time_start|productiontype|businesstype|quantity|resolution|resourceid|            time_end|
+----------------+--------------------+--------------+------------+--------+----------+----------+--------------------+
|             NO1| 2024-10-08 02:00:00|         Solar|         A04|  123.45|      PT1H|    RES001| 2024-10-08 03:00:00|
|             NO1|2025-10-09 11:58:...|         Hydro|         A04|  123.45|      PT1H|    RES001|2025-10-09 11:58:...|
+----------------+--------------------+--------------+------------+--------+----------+----------+--------------------+



In [3]:
from datetime import datetime

# Eksempeldata
data = [("NO1", datetime.utcnow(), "Hydro", "A04", 123.45, "PT1H", "RES001", datetime.utcnow())]
columns = ["meteringgridarea", "time_start", "productiontype", "businesstype", 
           "quantity", "resolution", "resourceid", "time_end"]

df = spark.createDataFrame(data, columns)

# Skriv til Cassandra
df.write \
    .format("org.apache.spark.sql.cassandra") \
    .mode("append") \
    .options(table="production_raw", keyspace="elhub_data") \
    .save()

print("✅ Data skrevet til Cassandra!")


/var/folders/db/2wck1jtn25v9rddfx0kw90qr0000gn/T/ipykernel_5916/1131737574.py:4: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  data = [("NO1", datetime.utcnow(), "Hydro", "A04", 123.45, "PT1H", "RES001", datetime.utcnow())]
25/10/09 13:58:05 WARN DeprecatedConfigParameter: spark.cassandra.connection.keep_alive_ms is deprecated (DSE 6.0.0) and has been automatically replaced with parameter spark.cassandra.connection.keepAliveMS. 
25/10/09 13:58:05 WARN DeprecatedConfigParameter: spark.cassandra.connection.keep_alive_ms is deprecated (DSE 6.0.0) and has been automatically replaced with parameter spark.cassandra.connection.keepAliveMS. 
25/10/09 13:58:05 WARN DeprecatedConfigParameter: spark.cassandra.connection.keep_alive_ms is deprecated (DSE 6.0.0) and has been automatically replaced with parameter spark.cassandra.connection.keep

✅ Data skrevet til Cassandra!


In [4]:
spark.read.format("org.apache.spark.sql.cassandra") \
    .options(table="production_raw", keyspace="elhub_data") \
    .load().show()


+----------------+--------------------+--------------+------------+--------+----------+----------+--------------------+
|meteringgridarea|          time_start|productiontype|businesstype|quantity|resolution|resourceid|            time_end|
+----------------+--------------------+--------------+------------+--------+----------+----------+--------------------+
|             NO1| 2024-10-08 02:00:00|         Solar|         A04|  123.45|      PT1H|    RES001| 2024-10-08 03:00:00|
|             NO1|2025-10-09 11:58:...|         Hydro|         A04|  123.45|      PT1H|    RES001|2025-10-09 11:58:...|
+----------------+--------------------+--------------+------------+--------+----------+----------+--------------------+



25/10/09 13:58:08 WARN DeprecatedConfigParameter: spark.cassandra.connection.keep_alive_ms is deprecated (DSE 6.0.0) and has been automatically replaced with parameter spark.cassandra.connection.keepAliveMS. 
25/10/09 13:58:08 WARN DeprecatedConfigParameter: spark.cassandra.connection.keep_alive_ms is deprecated (DSE 6.0.0) and has been automatically replaced with parameter spark.cassandra.connection.keepAliveMS. 
25/10/09 13:58:08 WARN DeprecatedConfigParameter: spark.cassandra.connection.keep_alive_ms is deprecated (DSE 6.0.0) and has been automatically replaced with parameter spark.cassandra.connection.keepAliveMS. 
